# Expo Analyse

### Bibs

In [332]:
import pandas as pd
import numpy as np

### Daten Import

In [333]:
art_hi = pd.read_pickle('../data/artikelhierarchie.pkl')
item_clu = pd.read_pickle('../data/item_cluster.pkl')
visits = pd.read_pickle('../data/visit_h.pkl')

In [334]:
art_hi.head(2)

,MARKE,ARTIKELNUMMER,ITEM_H_SK,SEASON,MKZ,KATEGORIE_BEZEICHNUNG,NOS,VEK,BEK,STAMM_VK
0,HUGO UNDERWEAR,28139351,63e265ae75ce388c07c5ece13769638c,149,423,Fashion,Nein,10.96,10.96,27.95
1,HUGO UNDERWEAR,28139351,63e265ae75ce388c07c5ece13769638c,149,423,Fashion,Nein,10.96,10.96,27.95


In [335]:
item_clu.sample(2)

,ITEM_H_SK,IS_DEALBUY,IS_NOS,CLUSTER_DIM1,CLUSTER_DIM2
19614984,ec3d902a7ce6fda5b65ff0055caf1042,False,False,KOLLEKTION,BESTANDSUEBERNAHME/RESTE
8569063,e5ebdfb32b12144009bb15c8d3f496bb,False,False,N.A.,N.A.


In [336]:
visits.head(2)

,visit_type,visit_h_sk,ot_bot
0,qVisit,b'\x11$\x11|\x84}}\x16\xee\x10\x01G\x02e/\x8f',False
1,qVisit,b'^?(\x85\\b\xfe\x80=(;Q\x9b\x04n0',False


### Explore 1
Tabellen kennen lernen für Bearbeitung 

shape all


In [337]:
print(f'art_hi: {art_hi.shape} \nitem_clu: {item_clu.shape} \nvisits: {visits.shape}')

art_hi: (97319, 10) 
item_clu: (19746564, 5) 
visits: (6080250, 3)


 `art_hi` Daten kennen lernen für Bearbeitung

In [338]:
art_hi.columns

Index(['MARKE', 'ARTIKELNUMMER', 'ITEM_H_SK', 'SEASON', 'MKZ',
       'KATEGORIE_BEZEICHNUNG', 'NOS', 'VEK', 'BEK', 'STAMM_VK'],
      dtype='object')

In [339]:
art_hi.isnull().sum()

MARKE                    0
ARTIKELNUMMER            0
ITEM_H_SK                0
SEASON                   0
MKZ                      0
KATEGORIE_BEZEICHNUNG    0
NOS                      0
VEK                      0
BEK                      0
STAMM_VK                 0
dtype: int64

 `item_clu` Daten kennen lernen für Bearbeitung

In [340]:
item_clu.isnull().sum()

ITEM_H_SK       0
IS_DEALBUY      0
IS_NOS          0
CLUSTER_DIM1    0
CLUSTER_DIM2    0
dtype: int64

In [341]:
item_clu.columns

Index(['ITEM_H_SK', 'IS_DEALBUY', 'IS_NOS', 'CLUSTER_DIM1', 'CLUSTER_DIM2'], dtype='object')

`visits` Daten kennen lernen für Bearbeitung 

In [342]:
visits.sample(2)

,visit_type,visit_h_sk,ot_bot
4612243,qVisit,"b'\xf7\x8ck\xf3\xb1\xab\x05\xb9`,\x1d\x05~\t\x...",False
778453,unknown,"b""\xce'\xc6\x07@]:\xec\xaa\x894q\xaa\xe7\x91\xf6""",False


In [343]:
visits.shape

(6080250, 3)

In [344]:
visits.isnull().sum()

visit_type    0
visit_h_sk    0
ot_bot        0
dtype: int64

In [345]:
visits['visit_type'].value_counts()

visit_type
unknown    5063521
qVisit      742829
bouncer     273900
Name: count, dtype: int64

### Transform Data 

### Tabellen Joinen
#### Cluster - Hierachi


Duplikate entfernen von ITEM_H_SK 

In [346]:
# Entferne Duplikate, behalte das erste Vorkommen
art_hi = art_hi.drop_duplicates(subset='ITEM_H_SK', keep='first')
item_clu = item_clu.drop_duplicates(subset='ITEM_H_SK', keep='first')

Überprüfung der Tabellen für Join

In [347]:
# Überprüfung, ob das Attribut 'ITEM_H_SK' in beiden DataFrames vorhanden ist
if 'ITEM_H_SK' not in art_hi.columns or 'ITEM_H_SK' not in item_clu.columns:
    raise ValueError("Das Attribut 'ITEM_H_SK' fehlt in einem der DataFrames.")

# Überprüfung auf Duplikate im Schlüsselattribut 'ITEM_H_SK' in beiden DataFrames
if art_hi['ITEM_H_SK'].duplicated().any():
    raise ValueError("Es gibt Duplikate im Attribut 'ITEM_H_SK' in art_hi.")
if item_clu['ITEM_H_SK'].duplicated().any():
    raise ValueError("Es gibt Duplikate im Attribut 'ITEM_H_SK' in item_clu.")

Inner Join für Überprüfung des gemeinsamen Attribut `ITEM_H_SK`

In [348]:
# Zusammenführen der DataFrames auf Basis von 'ITEM_H_SK'
art_item_inner = pd.merge(art_hi, item_clu, on='ITEM_H_SK', how='inner')

In [349]:
# Berechnet die Anzahl der Überschneidungen (gem. IETM_H_SK) der beiden DFs
anzahl_ueberschneidungen = len(art_item_inner)

print(f"Anzahl der Überschneidungen von 'ITEM_H_SK': {anzahl_ueberschneidungen}")

Anzahl der Überschneidungen von 'ITEM_H_SK': 15837


In [350]:
art_item_inner.sample(2)

,MARKE,ARTIKELNUMMER,ITEM_H_SK,SEASON,MKZ,KATEGORIE_BEZEICHNUNG,NOS,VEK,BEK,STAMM_VK,IS_DEALBUY,IS_NOS,CLUSTER_DIM1,CLUSTER_DIM2
4692,MARC O'POLO,59046851,7fdca1e4756fdcd8c25449cd91e002fa,149,231,Fashion,Ja,39.05,38.45,99.95,False,True,NOS,BESTANDSUEBERNAHME/RESTE
7575,TOMMY HILFIGER,34436612,b24b7789ccfe17b27750e1bea69248aa,149,226,Fashion,Nein,74.37,73.77,199.90,False,False,KOLLEKTION,BESTANDSUEBERNAHME/RESTE


#### art_item - views

hash des `visit_H_SK` entfernen

In [351]:
# Konvertiere Byte-Objekte in Hexadezimalform (visits_H_SK)
visits['visit_h_sk_hex'] = visits['visit_h_sk'].apply(lambda x: x.hex())

In [356]:
visits = visits[['visit_h_sk_hex', 'visit_type', 'ot_bot']]

,visit_h_sk_hex,visit_type,ot_bot
0,1124117c847d7d16ee10014702652f8f,qVisit,False
1,5e3f28855c62fe803d283b519b046e30,qVisit,False
2,ad5bb6ccb4f0262b813fe7af8778b7a2,qVisit,False
3,d33d9c97f8a7db1d882eb88423419937,qVisit,False
4,dbb9831b6406d6cdb7a026e909583c5d,qVisit,False
...,...,...,...
6080245,0e8506acb983d78506503b263b214f2e,unknown,True
6080246,28627db1190826f521669cdf6248cecf,unknown,True
6080247,4aa7c79c86c7e21153ba6f9498871f35,unknown,True
6080248,b1b6c70c17fb555ca40e062438152de1,unknown,True


In [ ]:
visits.sample(2)

4642988    bouncer
4403863     qVisit
Name: visit_type, dtype: object